### All In One Process

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from gtts import gTTS
#from playsound import playsound
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips
import pandas as pd
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import cv2

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# lang
if lang_folder == "English":
    lang = 'en'
elif lang_folder == "Turkish":
    lang = 'tr'
elif lang_folder == "German":
    lang = 'de'
elif lang_folder == "French":
    lang = 'fr'
elif lang_folder == "Spanish":
    lang = 'es'
elif lang_folder == "Arabic":
    lang = 'ar'
elif lang_folder == "Portuguese":
    lang = 'pt'
elif lang_folder == "Dutch":
    lang = 'nl'
elif lang_folder == "Italian":
    lang = 'it'
else:
    pass

print(f"{lang_folder}\n{lang}") 

In [ ]:
output_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Temp/Image Test"  # output path

#Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
audio_delay_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words/Delay/delay_audio.wav"

In [ ]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Temp/Image Test"

In [29]:
def text_convert_to_audio_same(word_group, out_folder_name, out_path, filename, language="en", speech_slow=True, delay_time = 0.5, delay_point = "both"):
    ''' 
    text_convert_to_audio_same(word_group, out_folder_name, out_path, filename, language="en", speech_slow=True, delay_time = 0.5, delay_point = "both")\n
    word_group is a string values. out_folder_name and out_path are path and folder name in str, filename is output result filename\n
    delay_time is in second and add delay to gttS audio result. delay point used for delay start, end or both\n 
    ex:\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    text_convert_to_audio(word_select_list, "Text_Audio", path, language=lang)
    '''
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    audio_delay = AudioFileClip(audio_delay_path).subclip(0,delay_time)
    if isfile(f"{result_path}/{filename}.mp3"):
        pass
    else:
        text_audio = gTTS(text=word_group, lang=language, slow=speech_slow)
        text_audio.save(f"{result_path}/{filename}.mp3")
        text_audio2 = AudioFileClip(f"{result_path}/{filename}.mp3")
        try:        
            if delay_point == "both":
                audio_combined = concatenate_audioclips([audio_delay,text_audio2,audio_delay])
            elif delay_point == "start":
                audio_combined = concatenate_audioclips([audio_delay,text_audio2])
            elif delay_point == "end":
                audio_combined = concatenate_audioclips([text_audio2,audio_delay])
            else:
                audio_combined = concatenate_audioclips([text_audio2])

            audio_combined.write_audiofile(f"{result_path}/{filename}.mp3", codec="mp3", bitrate="128k", fps=44100)
        except:
                print("Oops!", sys.exc_info()[0], "occurred.")

In [ ]:
def convert_image_to_jpg_crop_size(filename, data_folder_name, data_path, out_folder_name, out_path):
    '''
    convert_image_to_jpg(data_folder_name, data_path, out_folder_name, out_path)\n
    data_folder_name, data_path, out_folder_name and out_path are path and folder in string.\n
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    convert_image_to_jpg("Images", image_path, "Images_JPG", path)
    '''
    aspect_ratio_width = 16
    aspect_ratio_height = 9
    basewidth = 1600
    size_width = int(basewidth)
    size_heigth = int((basewidth/aspect_ratio_width)*aspect_ratio_height)
    size = (size_width,size_heigth)

    result_path = f"{out_path}/{out_folder_name}"
    data_folder_path = f"{data_path}/{data_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    
    if isfile(f"{result_path}/{filename}.jpg"):
        pass
    else:
        try:
            img_file = Image.open(f"{data_path}/{data_folder_name}/{filename}.jpg")
            img = img_file.convert("RGB")

            width, height = img.size
            height_var = int(height/aspect_ratio_height)
            width_var = int(width/aspect_ratio_width)

            if height_var == width_var:
                img_var = img.resize(size, Image.LANCZOS)
            elif width_var > height_var:
                crop_width = int(abs((width-(height_var*aspect_ratio_width))/2))
                box = (crop_width,0,width-crop_width,height)  # left, top, right, bottom
                img_crop = img.crop(box)  
                img_var = img_crop.resize(size, Image.LANCZOS)
            elif width_var < height_var:
                crop_height = int(abs((height-(width_var*aspect_ratio_height))/2))
                box = (0,crop_height,width,height-crop_height)  # left, top, right, bottom
                img_crop = img.crop(box)  # left, top, right, bottom
                img_var = img_crop.resize(size, Image.LANCZOS)
            else:
                img_var = img

            img_var.save(f"{result_path}/{filename}.jpg")

        except:
            print("Oops!", sys.exc_info()[0], "occurred.")

In [ ]:
def convert_pictures_to_video(filename, data_folder_name, data_path, out_folder_name, out_path, fps=1, time=2):
    '''
    convert_pictures_to_video(text_list, data_folder_name, data_path, out_folder_name, out_path, fps=1, time=2)\n
    text_list is list values. data_folder_name, data_path, out_folder_name and out_path are path\n
    and folder in string(str). others are cv2 parameter\n
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    convert_pictures_to_video(word_select_list, "Images_Crop_Size", image_path, "Image_Video", path, fps=1, time=2)
    '''
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    
    try:
        img = cv2.imread(f"{data_path}/{data_folder_name}/{filename}.jpg")
        height, width, layers = img.shape
        size=(width,height)
        frame_array = []
        for k in range (time):
            frame_array.append(img)
        result_path2 = f"{out_path}/{out_folder_name}/{filename}.mp4"
        out = cv2.VideoWriter(result_path2,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

        for i in range(len(frame_array)):
            out.write(frame_array[i])
        out.release()
    except:
        print("Oops!", sys.exc_info()[0], "occurred.")

In [33]:
def all_in_one(word_group_list, image_data_folder_name, image_data_path, out_path, language=lang):
    '''
    word_group_word_video_combine(word_group_list, video_data_folder_name, video_data_path, out_folder_name, out_path)
    word_group_list is word_group(sentence) list values. video_data_folder_name, video_data_path, out_folder_name and out_path are path and folder in string(str)\n
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    audio_image_video_path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    word_group_word_video_combine(df_deployment_file_sent_list, "Image_Video_Audio", audio_image_video_path, "200_Word_69_Youtube_Sentence", path)
    '''
    data_path = f"{out_path}/Data"
    result_path = f"{out_path}/Result"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    Path(result_path).mkdir(parents=True, exist_ok=True)
    for word_group in word_group_list:
        word_list = word_tokenize(word_group)
        if len(word_list)>3:
            filename = " ".join(word_list[0:3])
        else:
            filename = word_group
        text_convert_to_audio_same(word_group, "Wordgroup_MP3", data_path, filename, language=language, speech_slow=True, delay_time = 0.5, delay_point = "both")
        convert_image_to_jpg_crop_size(filename, image_data_folder_name, image_data_path, "Image_JPG_Crop", data_path)
        

In [34]:
word_group_list = ["adam","al","ama","ama","anne","artık","asla","aslında","aynı","baba","bak"]

In [35]:
all_in_one(word_group_list, "Images", image_path, output_path, language=lang)

MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Temp/Image Test/Data/Wordgroup_MP3/adam.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Temp/Image Test/Data/Wordgroup_MP3/al.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Temp/Image Test/Data/Wordgroup_MP3/ama.mp3


...

MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Temp/Image Test/Data/Wordgroup_MP3/baba.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Temp/Image Test/Data/Wordgroup_MP3/bak.mp3


MoviePy - Done.


In [21]:
delay_time = 0.5
wordgroup = "ben eve geldim"

In [22]:
audio_delay = AudioFileClip(audio_delay_path).subclip(0,delay_time)

In [25]:
text_audio = gTTS(text=wordgroup, lang=lang, slow="False")

In [26]:
audio_combined = concatenate_audioclips([audio_delay,text_audio,audio_delay])

AttributeError: 'gTTS' object has no attribute 'duration'

In [ ]:
df_deployment_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/\
Result/8-Adjust Word Group In Youtube Deploy Result/{lang_folder.lower().capitalize()}/Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_deployment_file = df_deployment_file.iloc[:,[0,1,2,3,4,5]]
df_deployment_file

In [ ]:
df_deployment_file_sent_list = df_deployment_file["search_string"].to_list()

In [ ]:
word_group_word_video_combine(df_deployment_file_sent_list, "Image_Video_Audio", audio_image_video_path, "200_Word_69_Youtube_Sentence", path)